<a href="https://colab.research.google.com/github/tlsdmswn01/Text_mining_project/blob/main/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
cd/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석

## 데이터 합치기

In [ ]:
df1=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링1000.xlsx')
df2=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링_title1000 (2).xlsx')
df3=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링_title800 (2).xlsx')
df4=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링600.xlsx')
df5=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링800.xlsx')
df6=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링_title400.xlsx')
df7=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링_title600.xlsx')
df8=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링_미완성800.xlsx')
df9=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링_미완성600.xlsx')
df10=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링_미완성1000.xlsx')
df11=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링_미완성400.xlsx')
df12=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링_미완성200.xlsx')
df13=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/감성분석/감성분석 라벨링 마지막.xlsx')






In [ ]:
df_list=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13]
analyze_df=pd.concat(df_list,ignore_index=True)

In [ ]:
analyze_df.info()

In [ ]:
analyze_df.dropna(inplace=True)

In [ ]:
analyze_df['감성라벨'].value_counts()
#analyze_df['감성라벨'].unique()

In [ ]:

fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(1,1,1)


ax1.pie(x=list(analyze_df['감성라벨'].value_counts()),labels=list(analyze_df['감성라벨'].unique()),autopct='%.2f%%')

ax1.set_title('analyze_df_dist')

fig.suptitle('Lable_data')

fig.tight_layout()

## 데이터 전처리

In [ ]:
analyze_df['감성라벨'].value_counts()

In [ ]:
analyze_df=analyze_df[analyze_df['감성라벨']!='-']

In [ ]:
analyze_df=analyze_df.dropna()

In [ ]:
analyze_df['감성라벨']=analyze_df['감성라벨'].astype('int')

In [ ]:
analyze_df.info()

In [ ]:
# 불용어 제거 - 한글과 공백 이외에 다른 글자들 제거(영어 제거하기)
analyze_df['제목'] = analyze_df['제목'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣!+?+❤️+ ]","")
print(analyze_df.isnull().sum())

## 테스트 데이터 바꿔보기  
- 1. SVM 2.LG

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(analyze_df['키워드'], analyze_df['감성라벨'] , test_size=0.2,random_state=112)


## 토큰화, 벡터화

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt # komoran, hannanum, kkma, mecab
from konlpy.tag import Kkma # komoran, hannanum, kkma, mecab

import re
import nltk
from nltk.stem import WordNetLemmatizer
# 불용어 사전 다운로드
stop_words=pd.read_csv('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/웹 크롤링 코드 및 데이터/stopwords.txt',sep='\t')

In [ ]:
# 토큰화
nltk.download('wordnet') #Lemmatization - 표제어 추출(통일화)
wl = WordNetLemmatizer()
okt=Okt()
def tokenizer(s):
    tk=okt.morphs(s)
    word_list=[]
    for word in tk:
        if word not in stop_words and len(word)!=1:
            word_list.append(wl.lemmatize(word))

    return word_list


In [ ]:
#벡터화
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=tokenizer,lowercase=False)
X_train=tfidf.fit_transform(x_train)


In [ ]:
tfidf.inverse_transform(X_train[0].toarray())

In [ ]:
X_test=tfidf.transform(x_test)

In [ ]:
tfidf.inverse_transform(X_test[0].toarray())

## 모델

In [ ]:
!pip install catboost

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier

### 로지스틱

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state=1004)
params = {'C': [15, 18, 19, 20, 22]}
grid_cv = GridSearchCV(log_reg, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(X_train, y_train)
print(grid_cv.best_params_)

In [ ]:
log_reg = LogisticRegression(C=19,random_state=1004)
log_reg.fit(X_train,y_train)

In [ ]:
print(log_reg.score(X_train, y_train)) # overfiting을 보려고 확인 -> train데이터서는 점수가 너무 좋은데 test 데이터에서는 성능이 좋지 않으면 overfiting발생
print(log_reg.score(X_test, y_test))

### 랜덤포레스트

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
folds = KFold(n_splits = 3, shuffle = True, random_state = 100)

In [ ]:
model=RandomForestClassifier()

#최적화할 파라미터 범위
rf_param = {'n_estimators': [100, 200],
             'max_depth': [20, 25],
             'min_samples_leaf' : [1, 2],
             'min_samples_split' : [2, 4]
             }

grid_rf = GridSearchCV(model, param_grid=rf_param, scoring = 'accuracy', cv=folds, n_jobs=-1)
grid_rf.fit(X_train, y_train)
print(grid_rf.best_params_)

In [ ]:
RF = RandomForestClassifier(n_estimators=100,max_depth=25,min_samples_leaf=1,min_samples_split=2,random_state=1004)
RF.fit(X_train,y_train)

In [ ]:
print(RF.score(X_train, y_train)) # overfiting을 보려고 확인 -> train데이터서는 점수가 너무 좋은데 test 데이터에서는 성능이 좋지 않으면 overfiting발생
print(RF.score(X_test, y_test))

### XGBoost

In [ ]:
model=XGBClassifier()

#최적화할 파라미터 범위
xg_param = {'learning_rate': [0.05, 0.1, 0.2],
             'n_estimator': [25, 50, 100],
             'max_depth': [6, 8, 10]
             }

grid_xg = GridSearchCV(model, param_grid=xg_param, scoring = 'accuracy', cv=folds, n_jobs=-1)
grid_xg.fit(X_train, y_train)
print(grid_xg.best_params_)

In [ ]:
XG = XGBClassifier(learning_rate=0.05,n_estimator=25,max_depth=8,random_state=1004)
XG.fit(X_train,y_train)

In [ ]:
print(XG.score(X_train, y_train)) # oveXGiting을 보려고 확인 -> train데이터서는 점수가 너무 좋은데 test 데이터에서는 성능이 좋지 않으면 oveXGiting발생
print(XG.score(X_test, y_test))

### SVM

In [ ]:
from sklearn.svm import SVC
linear_svm = SVC(kernel = 'linear', C = 1).fit(X_train, y_train)

In [ ]:
params={'C':[0.0001, 0.001, 0.01, 0.1, 1, 2,3,4,5,6]}

In [ ]:
grid_svm_linear = GridSearchCV(linear_svm, param_grid=params, scoring = 'accuracy', cv=folds, n_jobs=-1)
grid_svm_linear.fit(X_train, y_train)
print(grid_svm_linear.best_params_)

In [ ]:
C_list = [0.0001, 0.001, 0.01, 0.1, 1, 2,3,4,5,6]

# C값이 커질 수록 마진이 작아진다.
# C값을 변경해 가면서 각 모델들의 score를 확인하고자 한다.
# 한 번에 확인하기 위해서 C값을 리스트로 저장하고 for문을 이용하여 score를 구했다.

train_score = []
test_score = []
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(15, 16))

for C in C_list:
    linear_svm = SVC(kernel ='linear', C = C).fit(X_train, y_train)
    score_tr = linear_svm.score(X_train, y_train)
    score_te = linear_svm.score(X_test, y_test)
    train_score.append(score_tr)
    test_score.append(score_te)

    plt.figure(figsize=(10,5))
    plt.plot(train_score, 'go-')
    plt.plot(test_score, 'mo:')
    plt.xticks(range(len(C_list)),C_list)
    plt.xlabel('C')
    plt.ylabel('Accuracy')
    plt.legend(['train', 'test'])
    plt.show()


In [ ]:
linear_svm = SVC(kernel = 'linear', C =1,probability=True).fit(X_train, y_train)
print(linear_svm.score(X_train, y_train)) # ovefitting을 보려고 확인 -> train데이터서는 점수가 너무 좋은데 test 데이터에서는 성능이 좋지 않으면 ovelinear_svmiting발생
print(linear_svm.score(X_test, y_test))

In [ ]:
train_score = []
test_score = []

for C in C_list:
    linear_svm = SVC(kernel ='sigmoid', C = C).fit(X_train, y_train)
    score_tr = linear_svm.score(X_train, y_train)
    score_te = linear_svm.score(X_test, y_test)
    train_score.append(score_tr)
    test_score.append(score_te)

    plt.figure(figsize=(10,5))
    plt.plot(train_score, 'go-')
    plt.plot(test_score, 'mo:')
    plt.xticks(range(len(C_list)),C_list)
    plt.xlabel('C')
    plt.ylabel('Accuracy')
    plt.legend(['train', 'test'])
    plt.show()

In [ ]:
sigmoid_svm = SVC(kernel = 'sigmoid', C =1,probability=True).fit(X_train, y_train)
print(sigmoid_svm.score(X_train, y_train))
print(sigmoid_svm.score(X_test, y_test))

## 모델 평가

### ROC_curve 그래프 그리기

### AUC 면적 구하기
AUC는 ROC 커브 아래의 면적을 의미하는데, 0 ~ 1 사이의 값을 가지고,

1에 가까울수록 모델 분류 성능이 좋음을 의미

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
score1 = log_reg.predict_proba(X_test)[:, 1]
score2 = RF.predict_proba(X_test)[:, 1]
score3 = XG.predict_proba(X_test)[:, 1]
score5 = linear_svm.decision_function(X_test)
score6 = sigmoid_svm.decision_function(X_test)


fpr1, tpr1, _ = roc_curve(y_test, score1)
fpr2, tpr2, _ = roc_curve(y_test, score2)
fpr3, tpr3, _ = roc_curve(y_test, score3)
fpr5, tpr5, _ = roc_curve(y_test, score5)
fpr6, tpr6, _ = roc_curve(y_test, score6)


fig = plt.figure(figsize=(8,8))
fig.set_facecolor('white')
ax1 = fig.add_subplot()
ax1.plot(fpr1, tpr1, label='LR Score:{:.2f} AUC-{:.2f}'.format(log_reg.score(X_test, y_test),roc_auc_score(y_test, score1)))
ax1.plot(fpr2, tpr2, label='RF Score:{:.2f} AUC-{:.2f}'.format(RF.score(X_test, y_test),roc_auc_score(y_test, score2)))
ax1.plot(fpr3, tpr3, label='XG Score:{:.2f} AUC-{:.2f}'.format(XG.score(X_test, y_test),roc_auc_score(y_test, score3)))
ax1.plot(fpr5, tpr5, label='SVM_linear Score:{:.2f} AUC-{:.2f}'.format(linear_svm.score(X_test, y_test),roc_auc_score(y_test, score5)))
ax1.plot(fpr6, tpr6, label='SVM_sigmoid Score:{:.2f} AUC-{:.2f}'.format(sigmoid_svm.score(X_test, y_test),roc_auc_score(y_test, score6)))

ax1.plot([0, 1], [0, 1], color='red', label='Random Model')
ax1.legend()
plt.show()

## 최적화된 모델로 감성예측하기

In [ ]:
cd/content/drive/Shareddrives/텍스트 마이닝 프로젝트/군집분석

In [ ]:
df=pd.read_csv('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/군집분석/메인기사 10개 추출.csv')

In [ ]:
for i in range(10):
    input_text = df['key'][i]
    #입력 텍스트에 대한 전처리 수행
    input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(input_text)
    input_text = [" ".join(input_text)]
    # 입력 텍스트의 피처 벡터화
    st_tfidf = tfidf.transform(input_text)

    # 최적 감성 분석 모델에 적용하여 감성 분석 평가
    #st_predict = grid_cat.best_estimator_.predict(st_tfidf)
    st_predict = sigmoid_svm.predict(st_tfidf)

    #st_predict = log_reg.predict(st_tfidf)

    #예측 결과 출력
    if(st_predict == 0):
        print(df['title'][i])
        print('예측 결과: ->> 부정 감성')
        print('=====================================================')
    else :
        print(df['title'][i])
        print('예측 결과: ->> 긍정 감성')
        print('=====================================================')